# Seznam - Predicting the transaction volume

Seznam is a Czech company with a scope similar to Google. The purpose of this notebook is to analyze data from Seznam's wallet, predicting the transaction volume.

Summary:

- Prediction type: __Regression model__
- Domain: __E-commerce__
- Prediction target: __Transaction volume__ 
- Population size: __1,458,233__

_Author: Dr. Patrick Urbanke_

# Background

Seznam is a Czech company with a scope similar to Google. The purpose of this notebook is to analyze data from Seznam's wallet, predicting the transaction volume.

Since the dataset is in Czech, we will quickly translate the meaning of the main tables:

- *dobito*: contains data on prepayments into a wallet
- *probehnuto*: contains data on charges from a wallet
- *probehnuto_mimo_penezenku*: contains data on charges, from sources other than a wallet

The dataset has been downloaded from the [CTU Prague relational learning repository](https://relational.fit.cvut.cz/dataset/Seznam) (Motl and Schulte, 2015).

We will benchmark [getML](https://www.getml.com) 's feature learning algorithms against [featuretools](https://www.featuretools.com), an open-source implementation of the propositionalization algorithm, similar to getML's FastProp.

### A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

### Where is this running?

Your getML live session is running inside a docker container on [mybinder.org](https://mybinder.org/), a service built by the Jupyter community and funded by Google Cloud, OVH, GESIS Notebooks and the Turing Institute. As it is a free service, this session will shut down after 10 minutes of inactivity.

# Analysis

Let's get started with the analysis and set up your session:

In [1]:
import copy
import os
from urllib import request

import numpy as np
import pandas as pd
from IPython.display import Image
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline  

import featuretools
import getml

getml.set_project('seznam')




Connected to project 'seznam'


## 1. Loading data

### 1.1 Download from source

We begin by downloading the data:

In [2]:
conn = getml.database.connect_mariadb(
    host="relational.fit.cvut.cz",
    dbname="Seznam",
    port=3306,
    user="guest",
    password="relational"
)

conn

Connection(conn_id='default',
           dbname='Seznam',
           dialect='mysql',
           host='relational.fit.cvut.cz',
           port=3306)

In [3]:
def load_if_needed(name):
    """
    Loads the data from the relational learning
    repository, if the data frame has not already
    been loaded.
    """
    if not getml.data.exists(name):
        data_frame = getml.DataFrame.from_db(
            name=name,
            table_name=name,
            conn=conn
        )
        data_frame.save()
    else:
        data_frame = getml.data.load_data_frame(name)
    return data_frame

In [4]:
dobito = load_if_needed("dobito")
probehnuto = load_if_needed("probehnuto")
probehnuto_mimo_penezenku = load_if_needed("probehnuto_mimo_penezenku")

In [5]:
dobito

name,client_id,month_year_datum_transakce,sluzba,kc_dobito
role,unused_float,unused_string,unused_string,unused_string
0,7157857,2012-10-01,c,1045.62
1,109700,2015-10-01,c,5187.28
2,51508,2015-08-01,c,408.20
3,9573550,2012-10-01,c,521.24
4,9774621,2014-11-01,c,386.22
,...,...,...,...
554341,65283,2012-09-01,g,7850.00
554342,6091446,2012-08-01,g,31400.00
554343,1264806,2013-08-01,g,-8220.52


In [6]:
probehnuto

name,client_id,month_year_datum_transakce,sluzba,kc_proklikano
role,unused_float,unused_string,unused_string,unused_string
0,109145,2013-06-01,c,-31.40
1,9804394,2015-10-01,h,37.68
2,9803353,2015-10-01,h,725.34
3,9801753,2015-10-01,h,194.68
4,9800425,2015-10-01,h,1042.48
,...,...,...,...
1462073,98857,2015-08-01,NULL,153.86
1462074,95776,2015-09-01,NULL,153.86
1462075,98857,2015-09-01,NULL,153.86


In [7]:
probehnuto_mimo_penezenku

name,client_id,Month/Year,probehla_inzerce_mimo_penezenku
role,unused_float,unused_string,unused_string
0,3901,2012-08-01,ANO
1,3901,2012-09-01,ANO
2,3901,2012-10-01,ANO
3,3901,2012-11-01,ANO
4,3901,2012-12-01,ANO
,...,...,...
599381,9804086,2015-10-01,ANO
599382,9804238,2015-10-01,ANO
599383,9804782,2015-10-01,ANO


### 1.2 Prepare data for getML

getML requires that we define *roles* for each of the columns.

In [8]:
dobito.set_role("client_id", getml.data.roles.join_key)
dobito.set_role("month_year_datum_transakce", getml.data.roles.time_stamp)
dobito.set_role("sluzba", getml.data.roles.categorical)
dobito.set_role("kc_dobito", getml.data.roles.numerical)

dobito.set_unit("sluzba", "service")

dobito

name,month_year_datum_transakce,client_id,sluzba,kc_dobito
role,time_stamp,join_key,categorical,numerical
unit,"time stamp, comparison only",,service,
0,2012-10-01,7157857,c,1045.62
1,2015-10-01,109700,c,5187.28
2,2015-08-01,51508,c,408.2
3,2012-10-01,9573550,c,521.24
4,2014-11-01,9774621,c,386.22
,...,...,...,...
554341,2012-09-01,65283,g,7850
554342,2012-08-01,6091446,g,31400


In [9]:
probehnuto.set_role("client_id", getml.data.roles.join_key)
probehnuto.set_role("month_year_datum_transakce", getml.data.roles.time_stamp)
probehnuto.set_role("sluzba", getml.data.roles.categorical)
probehnuto.set_role("kc_proklikano", getml.data.roles.target)

probehnuto.set_unit("sluzba", "service")

probehnuto

name,month_year_datum_transakce,client_id,kc_proklikano,sluzba
role,time_stamp,join_key,target,categorical
unit,"time stamp, comparison only",,,service
0,2013-06-01,109145,-31.4,c
1,2015-10-01,9804394,37.68,h
2,2015-10-01,9803353,725.34,h
3,2015-10-01,9801753,194.68,h
4,2015-10-01,9800425,1042.48,h
,...,...,...,...
1462073,2015-08-01,98857,153.86,NULL
1462074,2015-09-01,95776,153.86,NULL


In [10]:
probehnuto_mimo_penezenku.set_role("client_id", getml.data.roles.join_key)
probehnuto_mimo_penezenku.set_role("Month/Year", getml.data.roles.time_stamp)

probehnuto_mimo_penezenku

name,Month/Year,client_id,probehla_inzerce_mimo_penezenku
role,time_stamp,join_key,unused_string
unit,"time stamp, comparison only",,
0,2012-08-01,3901,ANO
1,2012-09-01,3901,ANO
2,2012-10-01,3901,ANO
3,2012-11-01,3901,ANO
4,2012-12-01,3901,ANO
,...,...,...
599381,2015-10-01,9804086,ANO
599382,2015-10-01,9804238,ANO


In [11]:
split = getml.data.split.random(train=0.8, test=0.2)
split

,
0,train
1,train
2,train
3,test
4,train
,...


## 2. Predictive modeling

We loaded the data and defined the roles and units. Next, we create a getML pipeline for relational learning.

### 2.1 Define relational model

In [12]:
star_schema = getml.data.StarSchema(population=probehnuto, alias="population", split=split)

star_schema.join(
    probehnuto,
    on="client_id",
    time_stamps="month_year_datum_transakce",
    lagged_targets=True,
    horizon=getml.data.time.days(1),
)

star_schema.join(
    dobito,
    on="client_id",
    time_stamps="month_year_datum_transakce",
)

star_schema.join(
    probehnuto_mimo_penezenku,
    on="client_id", 
    time_stamps=("month_year_datum_transakce",  "Month/Year"),
)

star_schema

data model

  population:
    columns:
    - sluzba: categorical
    - client_id: join_key
    - kc_proklikano: target
    - month_year_datum_transakce: time_stamp

    joins:
    - right: 'probehnuto'
      on: (population.client_id, probehnuto.client_id)
      time_stamps: (population.month_year_datum_transakce, probehnuto.month_year_datum_transakce)
      relationship: 'many-to-many'
      horizon: 86400.0
      lagged_targets: True
    - right: 'dobito'
      on: (population.client_id, dobito.client_id)
      time_stamps: (population.month_year_datum_transakce, dobito.month_year_datum_transakce)
      relationship: 'many-to-many'
      lagged_targets: False
    - right: 'probehnuto_mimo_penezenku'
      on: (population.client_id, probehnuto_mimo_penezenku.client_id)
      time_stamps: (population.month_year_datum_transakce, probehnuto_mimo_penezenku.Month/Year)
      relationship: 'many-to-many'
      lagged_targets: False

  probehnuto:
    columns:
    - sluzba: categorical
    - client_id: join_key
    - kc_proklikano: target
    - month_year_datum_transakce: time_stamp

  dobito:
    columns:
    - sluzba: categorical
    - client_id: join_key
    - kc_dobito: numerical
    - month_year_datum_transakce: time_stamp

  probehnuto_mimo_penezenku:
    columns:
    - client_id: join_key
    - Month/Year: time_stamp
    - probehla_inzerce_mimo_penezenku: unused_string


container

  population
      subset   name            rows   type
  0   test     probehnuto    292833   View
  1   train    probehnuto   1169245   View

  peripheral
      name                           rows   type     
  0   probehnuto                  1462078   DataFrame
  1   dobito                       554346   DataFrame
  2   probehnuto_mimo_penezenku    599386   DataFrame

### 2.2 getML pipeline

<!-- #### 2.1.1  -->
__Set-up the feature learner & predictor__

We use the relboost algorithms for this problem. Because of the large number of keywords, we regularize the model a bit by requiring a minimum support for the keywords (`min_num_samples`).

In [13]:
mapping = getml.preprocessors.Mapping()

fast_prop = getml.feature_learning.FastProp(
    aggregation=getml.feature_learning.FastProp.agg_sets.All,
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,    
    sampling_factor=0.1,
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

__Build the pipeline__

In [14]:
pipe1 = getml.Pipeline(
    tags=['fast_prop'],
    data_model=star_schema.data_model,
    preprocessors=[mapping],
    feature_learners=[fast_prop],
    predictors=[predictor],
    include_categorical=True,
)

pipe1

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=True,
         loss_function=None,
         peripheral=['dobito', 'probehnuto', 'probehnuto_mimo_penezenku'],
         predictors=['XGBoostRegressor'],
         preprocessors=['Mapping'],
         share_selected_features=0.5,
         tags=['fast_prop'])

### 2.3 Model training

In [15]:
pipe1.check(star_schema.train)

Checking data model...


Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

Checking...
[========================================] 100%


INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and PROBEHNUTO__STAGING_TABLE_3 over 'client_id' and 'client_id', there are no corresponding entries for 0.022066% of entries in 'client_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and DOBITO__STAGING_TABLE_2 over 'client_id' and 'client_id', there are no corresponding entries for 2.250854% of entries in 'client_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and PROBEHNUTO_MIMO_PENEZENKU__STAGING_TABLE_4 over 'client_id' and 'client_id', there are no corresponding entries for 26.543966% 

In [16]:
pipe1.fit(star_schema.train)

Checking data model...


Staging...
[========================================] 100%


INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and PROBEHNUTO__STAGING_TABLE_3 over 'client_id' and 'client_id', there are no corresponding entries for 0.022066% of entries in 'client_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and DOBITO__STAGING_TABLE_2 over 'client_id' and 'client_id', there are no corresponding entries for 2.250854% of entries in 'client_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and PROBEHNUTO_MIMO_PENEZENKU__STAGING_TABLE_4 over 'client_id' and 'client_id', there are no corresponding entries for 26.543966% of entries in 'client_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.


Staging...
[======

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=True,
         loss_function=None,
         peripheral=['dobito', 'probehnuto', 'probehnuto_mimo_penezenku'],
         predictors=['XGBoostRegressor'],
         preprocessors=['Mapping'],
         share_selected_features=0.5,
         tags=['fast_prop', 'container-UIgTI7'])

url: http://localhost:1709/#/getpipeline/seznam/7NaN1W/0/

### 2.4 Model evaluation

In [17]:
pipe1.score(star_schema.test)



Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

FastProp: Building features...
[========================================] 100%




,date time,set used,target,mae,rmse,rsquared
0,2021-10-31 17:23:02,train,kc_proklikano,3090.3283,22460.03,0.8579
1,2021-10-31 17:24:48,test,kc_proklikano,3160.0238,24479.1592,0.7822


### 2.5 featuretools

In [18]:
population_train_pd = star_schema.train.population.to_pandas()
population_test_pd = star_schema.test.population.to_pandas()

In [19]:
population_train_pd["id"] = population_train_pd.index
population_test_pd["id"] = population_test_pd.index

In [20]:
probehnuto_pd = probehnuto.drop(probehnuto.roles.unused).to_pandas()
dobito_pd = dobito.drop(dobito.roles.unused).to_pandas()
probehnuto_mimo_penezenku_pd = probehnuto_mimo_penezenku.drop(probehnuto_mimo_penezenku.roles.unused).to_pandas()

In [21]:
def prepare_peripheral(peripheral_pd, train_or_test):
    """
    Helper function that imitates the behavior of 
    the data model defined above.
    """
    peripheral_new = peripheral_pd.merge(
        train_or_test[["id", "client_id", "month_year_datum_transakce"]],
        on="client_id"
    )

    peripheral_new = peripheral_new[
        peripheral_new["month_year_datum_transakce_x"] < peripheral_new["month_year_datum_transakce_y"]
    ]

    del peripheral_new["month_year_datum_transakce_y"]
    del peripheral_new["client_id"]

    return peripheral_new.rename({"month_year_datum_transakce_y": "month_year_datum_transakce"})

In [22]:
def prepare_probehnuto_mimo_penezenku(peripheral_pd, train_or_test):
    """
    Helper function that imitates the behavior of 
    the data model defined above.
    """
    peripheral_new = peripheral_pd.merge(
        train_or_test[["id", "client_id", "month_year_datum_transakce"]],
        on="client_id"
    )

    peripheral_new = peripheral_new[
        peripheral_new["Month/Year"] < peripheral_new["month_year_datum_transakce"]
    ]

    del peripheral_new["month_year_datum_transakce"]
    del peripheral_new["client_id"]

    return peripheral_new

In [23]:
dobito_train_pd = prepare_peripheral(dobito_pd, population_train_pd)
dobito_test_pd = prepare_peripheral(dobito_pd, population_test_pd)
dobito_train_pd

,sluzba,kc_dobito,month_year_datum_transakce_x,id
0,c,1045.62,2012-10-01,8636
1,c,1045.62,2012-10-01,44961
2,c,1045.62,2012-10-01,70626
3,c,1045.62,2012-10-01,104557
4,c,1045.62,2012-10-01,127731
...,...,...,...,...
17802333,g,31400.00,2012-08-01,1129230
17802334,g,31400.00,2012-08-01,1145288
17802335,g,31400.00,2012-08-01,1152364
17802336,g,31400.00,2012-08-01,1154444


In [24]:
probehnuto_train_pd = prepare_peripheral(probehnuto_pd, population_train_pd)
probehnuto_test_pd = prepare_peripheral(probehnuto_pd, population_test_pd)
probehnuto_train_pd

,sluzba,kc_proklikano,month_year_datum_transakce_x,id
5,c,-31.40,2013-06-01,1125178
6,c,-31.40,2013-06-01,1125209
7,c,-31.40,2013-06-01,1125218
10,c,-31.40,2013-06-01,1137830
11,c,-31.40,2013-06-01,1144770
...,...,...,...,...
44699290,None,1582.56,2012-08-01,1165076
44699293,e,87.92,2012-08-01,1165076
44699297,e,1827.48,2012-08-01,1165085
44699324,e,5086.80,2013-01-01,1166671


In [25]:
probehnuto_mimo_penezenku_train_pd = prepare_probehnuto_mimo_penezenku(probehnuto_mimo_penezenku_pd, population_train_pd)
probehnuto_mimo_penezenku_test_pd = prepare_probehnuto_mimo_penezenku(probehnuto_mimo_penezenku_pd, population_test_pd)
probehnuto_mimo_penezenku_train_pd

,Month/Year,id
0,2012-08-01,1014693
1,2012-08-01,1049636
3,2012-08-01,1077609
4,2012-09-01,1014693
5,2012-09-01,1049636
...,...,...
14257273,2015-09-01,635687
14257281,2015-09-01,8085
14257283,2015-09-01,145846
14257284,2015-09-01,477382


In [26]:
del population_train_pd["client_id"]
del population_test_pd["client_id"]

In [27]:
population_train_pd

,sluzba,kc_proklikano,month_year_datum_transakce,id
0,c,-31.40,2013-06-01,0
1,h,37.68,2015-10-01,1
2,h,725.34,2015-10-01,2
3,h,1042.48,2015-10-01,3
4,h,59.66,2015-10-01,4
...,...,...,...,...
1169240,None,153.86,2015-07-01,1169240
1169241,None,153.86,2015-08-01,1169241
1169242,None,153.86,2015-08-01,1169242
1169243,None,153.86,2015-09-01,1169243


In [28]:
dataframes_train = {
    "population" : (population_train_pd, "id"),
    "dobito": (dobito_train_pd, "index"),
    "probehnuto": (probehnuto_train_pd, "index"),
    "probehnuto_mimo_penezenku": (probehnuto_mimo_penezenku_train_pd, "index"),
}

In [29]:
dataframes_test = {
    "population" : (population_test_pd, "id"),
    "dobito": (dobito_test_pd, "index"),
    "probehnuto": (probehnuto_test_pd, "index"),
    "probehnuto_mimo_penezenku": (probehnuto_mimo_penezenku_test_pd, "index"),
}

In [30]:
relationships = [
    ("population", "id", "dobito", "id"),
    ("population", "id", "probehnuto", "id"),
    ("population", "id", "probehnuto_mimo_penezenku", "id"),
]

In [31]:
featuretools_train_pd = featuretools.dfs(
    dataframes=dataframes_train,
    relationships=relationships,
    target_dataframe_name="population")[0]

/home/patrick/.local/lib/python3.9/site-packages/featuretools/entityset/entityset.py:1567: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn("index {} not found in dataframe, creating new "


In [32]:
featuretools_test_pd = featuretools.dfs(
    dataframes=dataframes_test,
    relationships=relationships,
    target_dataframe_name="population")[0]

In [33]:
featuretools_train = getml.data.DataFrame.from_pandas(featuretools_train_pd, "featuretools_train")
featuretools_test = getml.data.DataFrame.from_pandas(featuretools_test_pd, "featuretools_test")

In [34]:
featuretools_train.set_role("kc_proklikano", getml.data.roles.target)
featuretools_train.set_role(featuretools_train.roles.unused_float, getml.data.roles.numerical)
featuretools_train.set_role(featuretools_train.roles.unused_string, getml.data.roles.categorical)

featuretools_train

name,kc_proklikano,sluzba,COUNT(dobito),MODE(dobito.sluzba),NUM_UNIQUE(dobito.sluzba),COUNT(probehnuto),MODE(probehnuto.sluzba),NUM_UNIQUE(probehnuto.sluzba),COUNT(probehnuto_mimo_penezenku),DAY(month_year_datum_transakce),MONTH(month_year_datum_transakce),WEEKDAY(month_year_datum_transakce),YEAR(month_year_datum_transakce),MODE(dobito.DAY(month_year_datum_transakce_x)),MODE(dobito.MONTH(month_year_datum_transakce_x)),MODE(dobito.WEEKDAY(month_year_datum_transakce_x)),MODE(dobito.YEAR(month_year_datum_transakce_x)),NUM_UNIQUE(dobito.DAY(month_year_datum_transakce_x)),NUM_UNIQUE(dobito.MONTH(month_year_datum_transakce_x)),NUM_UNIQUE(dobito.WEEKDAY(month_year_datum_transakce_x)),NUM_UNIQUE(dobito.YEAR(month_year_datum_transakce_x)),MODE(probehnuto.DAY(month_year_datum_transakce_x)),MODE(probehnuto.MONTH(month_year_datum_transakce_x)),MODE(probehnuto.WEEKDAY(month_year_datum_transakce_x)),MODE(probehnuto.YEAR(month_year_datum_transakce_x)),NUM_UNIQUE(probehnuto.DAY(month_year_datum_transakce_x)),NUM_UNIQUE(probehnuto.MONTH(month_year_datum_transakce_x)),NUM_UNIQUE(probehnuto.WEEKDAY(month_year_datum_transakce_x)),NUM_UNIQUE(probehnuto.YEAR(month_year_datum_transakce_x)),MODE(probehnuto_mimo_penezenku.DAY(Month/Year)),MODE(probehnuto_mimo_penezenku.MONTH(Month/Year)),MODE(probehnuto_mimo_penezenku.WEEKDAY(Month/Year)),MODE(probehnuto_mimo_penezenku.YEAR(Month/Year)),NUM_UNIQUE(probehnuto_mimo_penezenku.DAY(Month/Year)),NUM_UNIQUE(probehnuto_mimo_penezenku.MONTH(Month/Year)),NUM_UNIQUE(probehnuto_mimo_penezenku.WEEKDAY(Month/Year)),NUM_UNIQUE(probehnuto_mimo_penezenku.YEAR(Month/Year)),MAX(dobito.kc_dobito),MEAN(dobito.kc_dobito),MIN(dobito.kc_dobito),SKEW(dobito.kc_dobito),STD(dobito.kc_dobito),SUM(dobito.kc_dobito),MAX(probehnuto.kc_proklikano),MEAN(probehnuto.kc_proklikano),MIN(probehnuto.kc_proklikano),SKEW(probehnuto.kc_proklikano),STD(probehnuto.kc_proklikano),SUM(probehnuto.kc_proklikano)
role,target,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical
0,-31.4,c,1,c,1,13,d,1,0,1,6,5,2013,1,12,5,2012,1,1,1,1,1,8,0,2012,1,10,6,2,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1306.24,1306.24,1306.24,nan,nan,1306.24,351.68,155.7923,9.42,0.5817,79.3799,2025.3
1,37.68,h,0,NULL,NULL,0,NULL,NULL,0,1,10,3,2015,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,nan,nan,nan,nan,nan,0,nan,nan,nan,nan,nan,0
2,725.34,h,4,h,1,5,h,1,0,1,10,3,2015,1,5,0,2015,1,4,4,1,1,5,0,2015,1,5,5,1,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1036.2,614.655,257.48,0.5563,324.3624,2458.62,634.28,388.732,131.88,0.09478,205.8605,1943.66
3,1042.48,h,11,d,3,25,h,3,1,1,10,3,2015,1,10,2,2015,1,9,6,2,1,7,2,2015,1,12,7,2,1,9,1,2015,1,1,1,1,6280,4926.9455,2593.64,-0.8094,1472.2746,54196.4,5221.82,1869.0536,34.54,1.3198,1133.8706,46726.34
4,59.66,h,0,NULL,NULL,0,NULL,NULL,0,1,10,3,2015,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,nan,nan,nan,nan,nan,0,nan,nan,nan,nan,nan,0
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169240,153.86,NULL,1,c,1,9,NULL,0,0,1,7,2,2015,1,2,6,2015,1,1,1,1,1,2,0,2015,1,9,6,2,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2593.64,2593.64,2593.64,nan,nan,2593.64,188.4,158.7444,153.86,2.8961,11.2242,1428.7
1169241,153.86,NULL,8,c,1,36,NULL,0,0,1,8,5,2015,1,8,1,2013,1,7,5,4,1,1,0,20

In [35]:
featuretools_test.set_role("kc_proklikano", getml.data.roles.target)
featuretools_test.set_role(featuretools_test.roles.unused_float, getml.data.roles.numerical)
featuretools_test.set_role(featuretools_test.roles.unused_string, getml.data.roles.categorical)

featuretools_test

name,kc_proklikano,sluzba,COUNT(dobito),MODE(dobito.sluzba),NUM_UNIQUE(dobito.sluzba),COUNT(probehnuto),MODE(probehnuto.sluzba),NUM_UNIQUE(probehnuto.sluzba),COUNT(probehnuto_mimo_penezenku),DAY(month_year_datum_transakce),MONTH(month_year_datum_transakce),WEEKDAY(month_year_datum_transakce),YEAR(month_year_datum_transakce),MODE(dobito.DAY(month_year_datum_transakce_x)),MODE(dobito.MONTH(month_year_datum_transakce_x)),MODE(dobito.WEEKDAY(month_year_datum_transakce_x)),MODE(dobito.YEAR(month_year_datum_transakce_x)),NUM_UNIQUE(dobito.DAY(month_year_datum_transakce_x)),NUM_UNIQUE(dobito.MONTH(month_year_datum_transakce_x)),NUM_UNIQUE(dobito.WEEKDAY(month_year_datum_transakce_x)),NUM_UNIQUE(dobito.YEAR(month_year_datum_transakce_x)),MODE(probehnuto.DAY(month_year_datum_transakce_x)),MODE(probehnuto.MONTH(month_year_datum_transakce_x)),MODE(probehnuto.WEEKDAY(month_year_datum_transakce_x)),MODE(probehnuto.YEAR(month_year_datum_transakce_x)),NUM_UNIQUE(probehnuto.DAY(month_year_datum_transakce_x)),NUM_UNIQUE(probehnuto.MONTH(month_year_datum_transakce_x)),NUM_UNIQUE(probehnuto.WEEKDAY(month_year_datum_transakce_x)),NUM_UNIQUE(probehnuto.YEAR(month_year_datum_transakce_x)),MODE(probehnuto_mimo_penezenku.DAY(Month/Year)),MODE(probehnuto_mimo_penezenku.MONTH(Month/Year)),MODE(probehnuto_mimo_penezenku.WEEKDAY(Month/Year)),MODE(probehnuto_mimo_penezenku.YEAR(Month/Year)),NUM_UNIQUE(probehnuto_mimo_penezenku.DAY(Month/Year)),NUM_UNIQUE(probehnuto_mimo_penezenku.MONTH(Month/Year)),NUM_UNIQUE(probehnuto_mimo_penezenku.WEEKDAY(Month/Year)),NUM_UNIQUE(probehnuto_mimo_penezenku.YEAR(Month/Year)),MAX(dobito.kc_dobito),MEAN(dobito.kc_dobito),MIN(dobito.kc_dobito),SKEW(dobito.kc_dobito),STD(dobito.kc_dobito),SUM(dobito.kc_dobito),MAX(probehnuto.kc_proklikano),MEAN(probehnuto.kc_proklikano),MIN(probehnuto.kc_proklikano),SKEW(probehnuto.kc_proklikano),STD(probehnuto.kc_proklikano),SUM(probehnuto.kc_proklikano)
role,target,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,categorical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical,numerical
0,194.68,h,2,d,2,2,d,2,0,1,10,3,2015,1,9,1,2015,1,1,1,1,1,9,1,2015,1,1,1,1,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1296.82,777.15,257.48,nan,734.9244,1554.3,763.02,401.92,40.82,nan,510.6725,803.84
1,405.06,h,1,h,1,2,h,1,0,1,10,3,2015,1,8,5,2015,1,1,1,1,1,8,1,2015,1,2,2,1,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1296.82,1296.82,1296.82,nan,nan,1296.82,565.2,452.16,339.12,nan,159.8627,904.32
2,580.9,h,4,d,2,5,d,2,0,1,10,3,2015,1,9,1,2015,1,3,3,1,1,9,1,2015,1,4,4,1,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1296.82,1231.665,1036.2,-2.,130.31,4926.66,913.74,454.044,34.54,0.2893,328.7162,2270.22
3,106.76,h,0,NULL,NULL,0,NULL,NULL,0,1,10,3,2015,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,nan,nan,nan,nan,nan,0,nan,nan,nan,nan,nan,0
4,1927.96,h,15,d,2,21,d,2,0,1,10,3,2015,1,9,0,2015,1,10,6,2,1,9,0,2015,1,12,7,2,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,7784.06,1850.088,257.48,2.4789,1898.9207,27751.32,5199.84,1148.1933,25.12,1.8651,1342.4638,24112.06
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292828,153.86,NULL,5,c,1,36,d,2,0,1,4,2,2015,1,12,5,2013,1,4,4,3,1,8,5,2013,1,12,7,4,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1306.24,1045.62,31.4,-2.2358,566.9809,5228.1,351.68,150.3711,-31.4,-0.2998,56.2491,5413.36
292829,153.86,NULL,3,c,1,35,c,1,0,1,6,0,2015,1,4,1,2012,1,3,3,3,1,1,5,20

We train an untuned XGBoostRegressor on top of featuretools' features, just like we have done for getML's features.

Since some of featuretools features are categorical, we allow the pipeline to include these features as well. Other features contain NaN values, which is why we also apply getML's Imputation preprocessor.

In [36]:
data_model = getml.data.DataModel("population")

In [37]:
imputation = getml.preprocessors.Imputation()

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe2 = getml.Pipeline(
    tags=['featuretools'],
    data_model=data_model,
    preprocessors=[imputation],
    predictors=[predictor],
    include_categorical=True,
)

pipe2

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=True,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=['Imputation'],
         share_selected_features=0.5,
         tags=['featuretools'])

In [38]:
pipe2.fit(featuretools_train)

Checking data model...


Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

Checking...
[========================================] 100%


WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'day(month_year_datum_transakce)' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_string or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'mode(dobito.day(month_year_datum_transakce_x))' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_string or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'num_unique(dobito.day(month_year_datum_transakce_x))' in POPULATION__ST

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=True,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=['Imputation'],
         share_selected_features=0.5,
         tags=['featuretools'])

url: http://localhost:1709/#/getpipeline/seznam/OfTDnh/0/

In [39]:
pipe2.score(featuretools_test)



Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%




,date time,set used,target,mae,rmse,rsquared
0,2021-10-31 18:25:22,featuretools_train,kc_proklikano,5176.6759,31127.3969,0.7276
1,2021-10-31 18:25:24,featuretools_test,kc_proklikano,5183.3637,29730.0984,0.6876


### 2.6 Productionization

It is possible to productionize the pipeline by transpiling the features into production-ready SQL code. Please also refer to getML's `sqlite3` module.

In [40]:
# Creates a folder named seznam_pipeline containing
# the SQL code.
pipe1.features.to_sql().save("seznam_pipeline")

### 2.7 Discussion

For a more convenient overview, we summarize our results into a table.

Name                 | R-squared  | RMSE   | MAE
-------------------- | ---------- | -------| ----
getML: FastProp      |     78.22% | 24,480 | 3,160
featuretools         |     63.24% | 31,655 | 5,167

## 3. Conclusion

We have benchmarked getML against featuretools on a dataset related to online transactions. We have found that getML outperforms featuretools by a wide margin.

## References

Motl, Jan, and Oliver Schulte. "The CTU prague relational learning repository." arXiv preprint arXiv:1511.03086 (2015).

# Next Steps

This tutorial benchmarked getML against academic state-of-the-art algorithms from relational learning literature and getML's qualities with respect to categorical data.

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.